<a href="https://colab.research.google.com/github/ClickerDG/ml_dropout/blob/main/ml_dropout2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Завантажуємо інструменти та бібліотеки, Keras та TensorFlow

In [57]:
import tensorflow as tf
from tensorflow import keras

Завантажуємо дата-сет FashionMNIST та поділяємо дані на тестові, тренувальні та валідаційні

In [60]:
(train_images, train_labels),(test_images, test_labels) = keras.datasets.fashion_mnist.load_data()
size = 700
epochs_size = 600
train_labels = train_labels[:size]
test_labels = test_labels[:size]
train_images = train_images[:size] /  255.0
test_images = test_images[:size] / 255.0
validation_images = train_images[:size]
validation_labels = train_labels[:size]

Створюємо свою модель без шару Dropout

In [61]:
class CustomModel(keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.input_layer = keras.layers.Flatten(input_shape=(28,28))
        self.hidden1 = keras.layers.Dense(200, activation='relu')
        self.hidden2 = keras.layers.Dense(100, activation='relu')
        self.hidden3 = keras.layers.Dense(60, activation='relu')
        self.output_layer = keras.layers.Dense(10, activation='softmax')
    
    def call(self, input, training=None):
        input_layer = self.input_layer(input)
        hidden1 = self.hidden1(input_layer)
        hidden2 = self.hidden2(hidden1)
        hidden3 = self.hidden3(hidden2)
        output_layer = self.output_layer(hidden3)
        return output_layer

Завантажуємо розроблену модель та ініціалізуємо оптимізатори та гіперпараметри

In [62]:
model = CustomModel()
sgd = keras.optimizers.SGD(lr=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Тренуємо модель

In [63]:
model.fit(train_images, train_labels, epochs=epochs_size, validation_data=(validation_images, validation_labels))

Epoch 1/600
22/22 [==============================] - 1s 14ms/step - loss: 2.2047 - accuracy: 0.1586 - val_loss: 2.1052 - val_accuracy: 0.2743
Epoch 2/600
22/22 [==============================] - 0s 8ms/step - loss: 2.0392 - accuracy: 0.3700 - val_loss: 1.9501 - val_accuracy: 0.4500
Epoch 3/600
22/22 [==============================] - 0s 8ms/step - loss: 1.8885 - accuracy: 0.4814 - val_loss: 1.8017 - val_accuracy: 0.5457
Epoch 4/600
22/22 [==============================] - 0s 8ms/step - loss: 1.7377 - accuracy: 0.5700 - val_loss: 1.6492 - val_accuracy: 0.6357
Epoch 5/600
22/22 [==============================] - 0s 6ms/step - loss: 1.5904 - accuracy: 0.6443 - val_loss: 1.5005 - val_accuracy: 0.6557
Epoch 6/600
22/22 [==============================] - 0s 7ms/step - loss: 1.4474 - accuracy: 0.6486 - val_loss: 1.3823 - val_accuracy: 0.6571
Epoch 7/600
22/22 [==============================] - 0s 7ms/step - loss: 1.3137 - accuracy: 0.6629 - val_loss: 1.2495 - val_accuracy: 0.6714
Epoch 8/600


Протестуємо на тестовому дата-сеті

In [64]:
model.evaluate(test_images, test_labels)

22/22 [==============================] - 0s 2ms/step - loss: 1.1973 - accuracy: 0.7786


[1.1972922086715698, 0.7785714268684387]

Створюємо свою модель за шаром Dropout

In [66]:
class CustomModelDO(keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.input_layer = keras.layers.Flatten(input_shape=(28,28))
        self.hidden1 = keras.layers.Dense(200, activation='relu')
        self.hidden2 = keras.layers.Dense(100, activation='relu')
        self.hidden3 = keras.layers.Dense(60, activation='relu')
        self.output_layer = keras.layers.Dense(10, activation='softmax')
        self.dropout_layer = keras.layers.Dropout(rate=0.3)
    
    def call(self, input, training=None):
        input_layer = self.input_layer(input)
        input_layer = self.dropout_layer(input_layer)
        hidden1 = self.hidden1(input_layer)
        hidden1 = self.dropout_layer(hidden1, training=training)
        hidden2 = self.hidden2(hidden1)
        hidden2 = self.dropout_layer(hidden2, training=training)
        hidden3 = self.hidden3(hidden2)
        hidden3 = self.dropout_layer(hidden3, training=training)
        output_layer = self.output_layer(hidden3)
        return output_layer

Завантажуємо розроблену модель та ініціалізуємо оптимізатори та гіперпараметри

In [67]:
modelDO = CustomModelDO()
modelDO.compile(loss="sparse_categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

Тренуємо модель

In [68]:
modelDO.fit(train_images, train_labels, epochs=epochs_size, validation_data=(validation_images, validation_labels))

Epoch 1/600
22/22 [==============================] - 1s 14ms/step - loss: 2.3704 - accuracy: 0.1243 - val_loss: 2.2023 - val_accuracy: 0.3186
Epoch 2/600
22/22 [==============================] - 0s 6ms/step - loss: 2.2771 - accuracy: 0.1486 - val_loss: 2.1435 - val_accuracy: 0.4143
Epoch 3/600
22/22 [==============================] - 0s 7ms/step - loss: 2.2136 - accuracy: 0.1800 - val_loss: 2.0758 - val_accuracy: 0.4900
Epoch 4/600
22/22 [==============================] - 0s 8ms/step - loss: 2.1946 - accuracy: 0.2014 - val_loss: 2.0104 - val_accuracy: 0.5286
Epoch 5/600
22/22 [==============================] - 0s 8ms/step - loss: 2.1100 - accuracy: 0.2371 - val_loss: 1.9368 - val_accuracy: 0.5000
Epoch 6/600
22/22 [==============================] - 0s 7ms/step - loss: 2.0629 - accuracy: 0.2543 - val_loss: 1.8478 - val_accuracy: 0.5186
Epoch 7/600
22/22 [==============================] - 0s 8ms/step - loss: 2.0064 - accuracy: 0.2900 - val_loss: 1.7609 - val_accuracy: 0.5800
Epoch 8/600


Протестуємо на тестовому дата-сеті

In [69]:
modelDO.evaluate(test_images, test_labels)

22/22 [==============================] - 0s 2ms/step - loss: 0.5877 - accuracy: 0.8114


[0.5877384543418884, 0.8114285469055176]